In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema
from scipy.integrate import trapz
from scipy.optimize import Bounds, minimize

import meteostat

from geopy.geocoders import Nominatim

import requests
import json

import weatherapi

import plotly.io as pio


import warnings
pio.templates.default = "plotly"
warnings.filterwarnings('ignore')

# Define variables and formulas

In [2]:
kelvin = lambda x: x+273.15 # convert Celsius to Kelvin
celsius = lambda x: x-273.15 # convert Kelvin to Celsius
kWh = lambda x: x / 3600000 # convert Joule to kWh
Joule = lambda x: x * 3600000 # convert kWh to Joule

#freezing_zone = pd.Interval(left=-6, right=4) # temp between hp starts freezing and has to be thawed
freezing_zone = 4
freezing_factor = 1.2# factor which the efficiency coefficient is divided by in case of being in freezing zone
flow_max = 50
out_min = -10
room_temp = 21 # temperature at which room should be heated
ruecklauf = 25
ny = 0.4 #Gütegrad

temp_start_heat = 15 # temperature under which house has to be heated
house_energy = 200 # Watt / Kelvin

timeframe_h = 24 #hours
timeframe_s = timeframe_h * 3600 #seconds

#take Newton cooling data

#20 and 60 after some DIN-Norms
t_env = kelvin(20) #environmental temperature where water storage is located
t_0 = kelvin(60) #temperature at which water storage power is measured
#coef_h = 0.000625

#specifications of water storage (taken from real example)
storage_power = 200 #Watts
h2o_energy = 4200 #Joule / (kilogram * Kelvin)
storage_volume = 5000 #Liter = kilogram -> to be defined later on

#specifications of heatpump
hp_power = 7000 #Watts
hp_power_opt = 5000 #Watts -> Power in which heatpump works most efficiently (can be taken into calculation later)
hp_power_el_opt = 15000 #Watts -> Power in which heatpump works most efficiently (can be taken into calculation later)
max_heat_cap = 75 # temperature, up to which the heat pump can maximally heat

# other measures
groundwater = celsius(t_env)

flowtemp = lambda t: - (flow_max - room_temp) / (room_temp-out_min) *t + flow_max + out_min * (flow_max - room_temp) / (room_temp-out_min)
needed_house_energy = lambda t: np.maximum(0, room_temp - t) * house_energy # returns needed energy to heat house in Watt

#eta = lambda x: np.maximum(0, np.minimum(300, ny * kelvin(flowtemp(x)) / (kelvin(flowtemp(x))-kelvin(x))))
h = True
def eta(zu, ab = None): #theoretischer Wirkungsgrad
    if ab is None:
        ab = flowtemp(zu)
    return np.maximum(0, np.minimum(300, ny * kelvin(ab) / (kelvin(ab)-kelvin(zu))))

def eta_heating(zu, max_ab = None, min_ab = None ): #"gleitender" Wirkungsgrad -> da sich über temperaturerhöhung ändert
    try:
        if type(min_ab) == type(None):
            min_ab = ruecklauf
        if type(max_ab) == type(None):
            max_ab = flowtemp(zu)
    except:
        print("Error: function 'eta_heating'")
    return ny * (kelvin(zu) * np.log((kelvin(max_ab) - kelvin(zu)) / (kelvin(min_ab) - kelvin(zu))) + kelvin(max_ab) - kelvin(min_ab)) / (kelvin(max_ab) - kelvin(min_ab))

def eta_heating2(zu, max_ab = None, min_ab=None):
    try:
        if type(min_ab) == type(None):
            min_ab = ruecklauf
        if type(max_ab) == type(None):
            max_ab = flowtemp(zu)
    except:
        print("Error: function 'eta_heating'")

    return ny * ((kelvin(max_ab) - kelvin(min_ab)) / (kelvin(max_ab) - kelvin(min_ab) - kelvin(zu) * np.log(kelvin(max_ab) / kelvin(min_ab))))

def eta_freezing_adj(zu, max_ab = None, min_ab = None, heating =  False): # if heating = True, then use eta_heating
    try:
        if type(min_ab) == type(None):
            min_ab = ruecklauf
        if type(max_ab) == type(None):
            max_ab = flowtemp(zu)
    except:
        print("Error: function 'eta_heating'")

    if heating == True:
        e = eta_heating2(zu, max_ab, min_ab)
    else:
        e = eta(zu, max_ab)
    #print(type(eta))
    if type(e) == np.float64:
        if zu < 4:
            e = e / freezing_factor
    else:
        e[zu < 4] = e[zu < 4] / 1.2

    return e

def newton_cooling(t):
    return t_env + (t_0 - t_env) * np.exp(-coef_h * t) # = temperature at time t

def newton_cooling_diff(t):
    return coef_h * (t_env - newton_cooling(t)) # = temperature loss rate at time t

def newton_cooling_t_0(temp_t, t=7*3600): #enter in Kelvin
    #return ((temp_t - t_env) * np.exp((coef_h * t).T)).T + t_env #returns t_0 for fix temperature after time t
    return (temp_t - t_env) * np.exp((coef_h * t)) + t_env #returns t_0 for fix temperature after time t

def newton_cooling_t(temp_t):#, coef_h = coef_h):
    return -np.log((temp_t - t_env) / (t_0 - t_env)) / coef_h #returns how much time it takes for t_0 to cool down to wished temperature temp_t

def storage_power_temp(temp):
    return  -h2o_energy * storage_volume * newton_cooling_diff(newton_cooling_t(temp))#returns how much power in Watt is needed to keep temperature at temp level

def get_d_t():
    return storage_power / (h2o_energy * storage_volume) # returns heat loss in kelvin per second

def get_coef_h(d_t):
    return d_t / np.abs(t_env - t_0) #returns coefficient at which heat decreases in storage

def get_coef_h_adj(temp, flow, t):
    return - np.log((flow - t_env) / (temp - t_env)) / t # returns adjusted coef_h for given flow, overheat_temp and time after which flow should be reached

def storage_power_temp_adj(temp, flow, t):
    return h2o_energy * storage_volume * (coef_h - get_coef_h_adj(temp, flow, t)) * (temp - t_env) # returns needed power to keep temperature in storage and decrease at given rate

def get_needed_storage_vol(temp_diff, kWh):
    return (kWh * 3600000 )/ (temp_diff * h2o_energy) #returns waterstorage volume to store #kWh with given temp difference

d_t = get_d_t()
coef_h = get_coef_h(d_t)

integral = lambda x, y: trapz(y, x) # returns the estimated integral for data points x and y
integral_avg = lambda x, y: integral(x, y) / (x.shape[0] - 1) # returns the average of the integral for points x and y

heat = lambda ground_temp, flowtemp, eta_t_0: (flowtemp - ground_temp) * h2o_energy * storage_volume / eta_t_0 # energy needed to heat water from ground temp to flow with given eta (in Joule)
overheat = lambda time, flowtemp, eta_t_0: (newton_cooling_t_0(kelvin(flowtemp), t = time*3600) - 273.15 - flowtemp) * h2o_energy * storage_volume / eta_t_0 # energy needed to heat water storage from flowtemp to t_0
keepheat = lambda time, flowtemp, eta_avg: storage_power_temp(kelvin(flowtemp))*3600*time / eta_avg # energy needed to keep heat in water storage on flowtemp with changing eta
def over_keep_mix(time, flowtemp, eta_t_0, eta_avg): # get the minimum of a mix of overheating and keeping the heat

    try:
        if time == 0: #if time equals 0, then there is no optimum that can be computed
            return flowtemp, heat(groundwater, flowtemp, eta_t_0), 0

        def target_function(temp): #returns the energy needed for heating, overheating and keeping the heat level in Joule
            over = (temp - flowtemp) * h2o_energy * storage_volume / eta_t_0
            keep = storage_power_temp_adj(kelvin(temp), kelvin(flowtemp), time * 3600) * 3600 * time / eta_avg
            h = heat(groundwater, flowtemp, eta_t_0)

            return over + keep + h

        lb = flowtemp
        ub = max(flowtemp, (celsius(newton_cooling_t_0(kelvin(flowtemp), time*3600))))

        cons = Bounds(lb = lb, ub= ub)
        #print(cons)
        min = minimize(fun = target_function, x0=(flowtemp+0.0001, ), bounds=cons)

        return min.x[0], min.fun[0], storage_power_temp_adj(kelvin(min.x[0]), kelvin(flowtemp), time * 3600) #TODO insert what is returned
    except:
        return np.nan, np.nan, np.nan

over_keep_mix_vectorized = np.vectorize(over_keep_mix)

energy_total_overheat = lambda ground_temp, flowtemp, eta_t_0, time: heat(ground_temp, flowtemp, eta_t_0) + overheat(time, flowtemp, eta_t_0) # just consider overheating
energy_total_keepheat = lambda ground_temp, flowtemp, eta_t_0, eta_avg, time: heat(ground_temp, flowtemp, eta_t_0) + keepheat(time, flowtemp, eta_avg) # just consider keeping heat

# Add calculated Data to Dataframe

In [3]:
def add_data(df):
    df['efficiency'] = eta_freezing_adj(df.temperature, heating=h)
    df['flow_temp'] = flowtemp(df.temperature)
    df['energy_needed_water'] = needed_house_energy(df.temperature)
    df['energy_needed_elec'] = needed_house_energy(df.temperature) / df.efficiency
    df['time'] = df.date.dt.time
    df['freezing_zone'] = df['temperature'] < freezing_zone

    return df

def get_local_extrema(df): #get local minima and maxima in function
    check_interval = 10

    df['local_min'] = df.iloc[argrelextrema(df.temperature.values, np.less_equal, order=check_interval)[0]]['temperature']
    df['local_max'] = df.iloc[argrelextrema(df.temperature.values, np.greater_equal, order=check_interval)[0]]['temperature']
    df['local_max_flow'] = df.iloc[argrelextrema(df.flow_temp.values, np.greater_equal, order=check_interval)[0]]['flow_temp']

    #delete doubled high and low points -> take only the later ones
    df.local_min[(pd.notna(df.local_min - df.local_min.shift(-1))) | (pd.notna(df.local_min - df.local_min.shift(-2))) | (pd.notna(df.local_min - df.local_min.shift(-3)))] = np.nan
    df.local_max[(pd.notna(df.local_max - df.local_max.shift(1))) | (pd.notna(df.local_max - df.local_max.shift(2)))| (pd.notna(df.local_max - df.local_max.shift(3)))] = np.nan

    # high and low points have to alternate
    local_max_index = df[pd.notna(df.local_max)].index # get indexes of local max
    local_min_index = df[pd.notna(df.local_min)].index # get indexes of local min

    index = np.sort(np.concatenate((local_min_index.values, local_max_index.values))) # concat and sort both indexes
    index_pd = pd.Series(np.isin(index, local_min_index))

    index = index[(index_pd - index_pd.shift(1)) != 0] # when two max or two min indexes occur, shifting and taking the difference results in 0

    local_min_index = local_min_index[np.isin(local_min_index, index)] # take only remaining indexes
    local_max_index = local_max_index[np.isin(local_max_index, index)]

    if local_max_index[0] < local_min_index[0]: #reassure that period counting starts with period_min -> heating period comes before deheating
        local_max_index = local_max_index[1:]

    for i, index in enumerate(local_min_index):
        df.loc[df.index > index, 'period_min'] = i

    for i, index in enumerate(local_max_index):
        df.loc[df.index >= index, 'period_max'] = i

    return df

# Import data from dwd

(Deutscher Wetterdienst)

In [43]:
def read_dat(path):
    data = pd.read_table(path, skiprows=30)
    col = data.columns[0]
    data = data[col].str.split(expand=True)
    data.columns = col.split()
    data = data.loc[1:]

    data.MM = data.MM.str.zfill(2)
    data.DD = data.DD.str.zfill(2)
    data.HH = data.HH.str.zfill(2)
    index_24 = data.HH[data.HH == '24'].index
    data.HH[data.HH == '24'] = '23'
    data['date'] = pd.to_datetime('2022-' + data.MM.astype(str) + '-' + data.DD.astype(str) + ' ' + data.HH.astype(str) + ':00', format = '%Y-%m-%d %H:%M')

    data.loc[index_24, 'date'] = data.loc[index_24].date + pd.to_timedelta(1, 'h')
    data.rename({'t': 'temperature'}, axis=1, inplace=True)
    data = data[['date','temperature']]
    data.temperature = data.temperature.astype(float)
    data = data.sort_values(by='date')
    return data

data_temp = read_dat('data/TRY_data/TRY2015_490148084221_Jahr.dat')
data_temp = data_temp[(data_temp.date >= pd.to_datetime('2022-01-01')) & (data_temp.date <= pd.to_datetime('2022-04-30'))]
#data_temp = data_temp[(data_temp.date.dt.month >= 10) | (data_temp.date.dt.month <= 3)]
data_temp.reset_index(drop = True, inplace=True)

In [4]:


#data_temp = pd.read_table('data/stundenwerte_TU_VS_akt/produkt_tu_stunde_20201216_20220618_05229.txt', sep=';') #Villingen-Schwenningen
data_temp = pd.read_table('data/weatherstationdata/produkt_tu_stunde_19480101_20081031_02522.txt', sep=';') #Karlsruhe
#data_temp = pd.read_table('data/weatherstationdata/produkt_tu_stunde_19500101_20211231_05792.txt', sep=';') #Zugspitze
#data_temp = pd.read_table('data/stundenwerte_TU_05100_19410101_20211231_hist/produkt_tu_stunde_19410101_20211231_05100.txt', sep=';') #Trier?
data_temp = data_temp.drop(['eor', 'QN_9', 'RF_TU'], axis=1) # drop end-of-row indicator
data_temp.rename(columns = {'TT_TU':'temperature', 'MESS_DATUM': 'date', }, inplace = True)
data_temp.date = pd.to_datetime(data_temp.date.astype(str), format='%Y%m%d%H')
data_temp[data_temp == -999] = np.nan # rename missing values to numpy nan
data_temp = data_temp[(data_temp.date >= pd.to_datetime('2007-10-01')) & (data_temp.date <= pd.to_datetime('2008-04-30'))]
#data_temp = data_temp[(data_temp.date.dt.month >= 10) | (data_temp.date.dt.month <= 3)]
data_temp.reset_index(drop = True, inplace=True)

In [5]:
data_temp = add_data(data_temp)
data_temp = get_local_extrema(data_temp)

In [45]:
px.line(data_temp, x='date', y='temperature', title='temperature')

# Visualizations for temperature

In [8]:
px.histogram(data_temp.freezing_zone, title='Number of hours in Freezing zone (<4°C)')

In [10]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = data_temp.date, y = data_temp.temperature, name='temperature'), secondary_y = False)
fig.add_trace(go.Scatter(x = data_temp.date, y = data_temp.local_min, mode='markers', name='local_minima'), secondary_y = False)
fig.add_trace(go.Scatter(x = data_temp.date, y = data_temp.local_max, mode='markers', hovertext=data_temp.period_max, name='local_maxima'), secondary_y = False)
fig.update_layout(dict1={'title': 'temperature with high and low points'})
fig.show()

# Mean and STD for Diurnal Temperature Range

In [46]:
temp_min = data_temp.groupby(by='period_max').local_min.min().reset_index()
temp_max = data_temp.groupby(by='period_min').local_max.max().reset_index()

temp_ext = pd.merge(left=temp_max, right=temp_min, left_on='period_min', right_on='period_max')
temp_ext

,period_min,local_max,period_max,local_min
0,0.0,2.0,0.0,-1.7
1,1.0,0.0,1.0,-7.1
2,2.0,-1.0,2.0,-7.4
3,3.0,6.1,3.0,2.1
4,4.0,8.9,4.0,3.3
...,...,...,...,...
100,100.0,17.8,100.0,11.2
101,101.0,20.9,101.0,10.7
102,102.0,19.0,102.0,9.3
103,103.0,14.2,103.0,6.1


In [47]:
(temp_ext.local_max - temp_ext.local_min).mean()

6.67904761904762

In [48]:
(temp_ext.local_max - temp_ext.local_min).std()

3.325273130427242

In [224]:
data_temp

,STATIONS_ID,date,temperature,efficiency,flow_temp,energy_needed_water,energy_needed_elec,time,freezing_zone,local_min,local_max,local_max_flow,period_min,period_max
0,5229,2022-01-01 00:00:00,1.2,3.282978,39.522581,3960.0,1206.221925,00:00:00,True,NaN,NaN,NaN,NaN,NaN
1,5229,2022-01-01 01:00:00,1.4,3.313202,39.335484,3920.0,1183.145440,01:00:00,True,NaN,NaN,NaN,NaN,NaN
2,5229,2022-01-01 02:00:00,0.6,3.195638,40.083871,4080.0,1276.740236,02:00:00,True,NaN,NaN,NaN,NaN,NaN
3,5229,2022-01-01 03:00:00,-0.3,3.073299,40.925806,4260.0,1386.132545,03:00:00,True,NaN,NaN,NaN,NaN,NaN
4,5229,2022-01-01 04:00:00,-0.6,3.034650,41.206452,4320.0,1423.557778,04:00:00,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,5229,2022-03-30 20:00:00,5.8,4.995367,35.219355,3040.0,608.563904,20:00:00,False,NaN,NaN,NaN,82.0,82.0
2133,5229,2022-03-30 21:00:00,5.7,4.966216,35.312903,3060.0,616.163291,21:00:00,False,NaN,NaN,NaN,82.0,82.0
2134,5229,2022-03-30 22:00:00,5.4,4.880826,35.593548,3120.0,639.236101,22:00:00,False,NaN,NaN,35.593548,82.0,82.0
2135,5229,2022-03-30 23:00:00,5.4,4.880826,35.593548,3120.0,639.236101,23:00:00,False,NaN,NaN,35.593548,82.0,82.0


# Weather forecast

In [225]:
def get_weather_forecast(address, days=4, key = '676ea42d0beb4636b96160506221607'):

    # Get coordinates for adress
    geolocator = Nominatim(user_agent="user")
    location = geolocator.geocode(address)

    # Get nearby weather stations
    stations = meteostat.Stations()
    stations = stations.nearby(location.latitude, location.longitude)
    station = stations.fetch(1)

    # Print DataFrame
    print(station)
    weather_data = requests.get(f'http://api.weatherapi.com/v1/forecast.json?key={key}&q={address}&days={days}&aqi=no&alerts=no')

    f = pd.DataFrame.from_dict(json.loads(weather_data.content.decode())['forecast']['forecastday'])
    forecast = pd.json_normalize(pd.json_normalize(f['hour']).stack())

    forecast.time = pd.to_datetime(forecast.time, )
    forecast = forecast[['time', 'temp_c']]

    forecast.columns = ['date', 'temperature']
    return forecast

In [226]:
forecast = get_weather_forecast('Karlsruhe', days=4)
forecast.temperature = forecast.temperature - 17

forecast = add_data(forecast)
forecast = get_local_extrema(forecast)
forecast = forecast[forecast.date > pd.to_datetime('now')]

            name country region    wmo  icao  latitude  longitude  elevation  \
id                                                                             
10727  Karlsruhe      DE     BW  10727  EDIL   49.0333     8.3667      112.0   

            timezone hourly_start hourly_end daily_start  daily_end  \
id                                                                    
10727  Europe/Berlin   1948-01-01 2008-11-02  1876-01-01 2008-11-01   

      monthly_start monthly_end     distance  
id                                            
10727    1876-01-01  2008-01-01  3975.721939  


In [227]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = forecast.date, y = forecast.temperature), secondary_y=False)
fig.add_trace(go.Scatter(x = forecast.date, y = forecast.local_min, mode='markers', name='local_minima'), secondary_y = False)
fig.add_trace(go.Scatter(x = forecast.date, y = forecast.local_max, mode='markers', hovertext=forecast.period_max, name='local_maxima'), secondary_y = False)
#fig.add_trace(go.Scatter(x = forecast.date, y = forecast.energy_needed_water, mode='lines', name='energy_needed_water'), secondary_y= True)
#fig.add_trace(go.Scatter(x = forecast.date, y = forecast.energy_needed_water_integ, mode='lines', name='energy_needed_water_integ'), secondary_y= True)
#fig.add_trace(go.Scatter(x = forecast.date, y = forecast.en_produced_total, mode='lines', name='en_produced_total'), secondary_y= True)
#fig.add_trace(go.Scatter(x = forecast.date, y = forecast.en_produced_hourly, mode='lines', name='en_produced_hourly'), secondary_y= True)
#fig.add_trace(go.Scatter(x = forecast.date, y = forecast.eta_hp_heating_nextmin, mode='lines', name='eta_nextmin'), secondary_y= False)
#fig.add_trace(go.Scatter(x = forecast.date, y = forecast.eta_hp_heating_nextmin_freezing_adjusted, mode='lines', name='eta_nextmin_freezing_adj.'), secondary_y= False)
fig.add_hline(y = 4)
fig.update_layout(title= 'temperature Simulation (From forecast)')
fig.show()

# Optimization problem 1

simplified version: period_lim = None
assumptions:
- Unlimited Storage
- Heating temperature is equal for every temperature (max_ab_i = x for all i)
- Efficiency only depends on environmental Temperature
- ruecklauf is equal for each temperature outside -> whenever storage temperature is higher, it is sufficient for heating and can be adjusted by water flow



extended version: period_lim = periods restrict optimization on

In [228]:
def heating_opt(df, period_lim = None, period = None, fl = None):
    #df_per = df[(df.period_max == p) | (df.period_min == p)]
    if type(period) != type(None):
        df_per = df[(df.period_max == period) | (df.period_min == period)]
    else:
        df_per = df.copy()
    per_min = df[df.period_min == period] #period from miminum to minimum
    per_max = df[df.period_max == period] #period from max to max

    per = df_per.shape[0] #number of hours in period
    M_timedelta = np.empty((per, per))

    for i in range(per): # build timedelta matrix
        M_timedelta[i, range(0, i+1)] = np.flip(np.linspace(0, i, i+1))
    M_timedelta = M_timedelta.T
    flow = df_per.flow_temp.values

    M_flow = celsius(newton_cooling_t_0(kelvin(flow), M_timedelta * 3600))
    M_flow = np.triu(M_flow) # needed heating temperature when heating in period i and using the heat in period j (i X j - Matrix)
    M_temperature = np.tile(df_per.temperature.values, (len(df_per.temperature.values),1)).T

    M_eta = eta_freezing_adj(zu=M_temperature, max_ab=M_flow)#, min_ab=M_flow)

    if type(period_lim) != type(None):

        M_flow_max = celsius(newton_cooling_t_0(kelvin(fl), t=M_timedelta * 3600))
        M_eta_heating = eta_freezing_adj(zu = M_temperature, max_ab=M_flow_max, heating=True)
        M_eta_heating = np.triu(M_eta_heating, k=1)
        M_eta = np.tril(M_eta)

        M_eta = M_eta_heating + M_eta
        M_eta = np.triu(M_eta) # upper diagonal
        M_eta = np.tril(M_eta, k = period_lim)

    else:
        M_eta = np.triu(M_eta) # upper diagonal

    P_w_sum = df_per.energy_needed_water.values # needed energy in water for each hour
    P_e_max = np.full((per, ), hp_power_el_opt) # maximal available energy (electricity) per hour -> e.g. 2500W
    P_e_available = P_e_max.copy() # at each period still available 'capacity' in electricity for future hours

    P_e = np.zeros((per, per)) # Matrix with used energy (e) from each period for each period that is optimized
    P_e_a = np.empty((per, per)) # Matrix to later calculate P_e

    for j in range(per):
        P_w_j = P_w_sum[j] # energy needed (w) in hour j

        M_eta_j = M_eta[: , j] # efficiencies in each hour
        eta_sorted_j_index = np.flip(np.argsort(M_eta_j)) # get indexes of highes values in M_eta_j
        eta_sorted_j = M_eta_j[eta_sorted_j_index] # get highest values of M_eta_j (descending order)

        P_w_available_j = P_e_available[eta_sorted_j_index] * eta_sorted_j # available energy (w) for hour j in descending cost order

        P_w_available_j_cumsum = P_w_available_j.cumsum() # get cumsum for available P_w (descending cost order)
        P_w_available_j_cumsum_shift = P_w_available_j_cumsum - P_w_available_j # shifted cumsum

        # calculate which P_w_available_j values are used fully and which only partly (only one will be used partly)
        cumsum_bin = P_w_available_j_cumsum < P_w_j # binary values if cumsum is smaller than needed energy
        cumsum_shift_bin = P_w_available_j_cumsum_shift < P_w_j # binary values if cumsum_shift is smaller than needed energy

        idx = (cumsum_bin == False) & (cumsum_shift_bin == True) # only True for partly used P_w_j
        percent = (P_w_available_j_cumsum[idx] -P_w_j)  / P_w_available_j[idx] # get remaining share of partly used P_w_j
        P_e_percent_remaining = np.ones((per, )) # first, all remaining values are set to 1 (=100%)
        P_e_percent_remaining[cumsum_bin] = 0 # set all fully used P_w_j to 0
        P_e_percent_remaining[idx] = percent # set partly used to calculated percentage

        P_e_percent_remaining_sorted_orig = P_e_percent_remaining[np.argsort(eta_sorted_j_index)] # set remaining shares to original order
        P_e_available = P_e_available * P_e_percent_remaining_sorted_orig # get new P_e_available
        P_e_a[:,j] = P_e_available

    P_e_a = np.triu(P_e_a) # take only upper diagonal
    P_e[:,0] = P_e_a[:,0]
    for j in range(1, per):
        P_e[:,j] = np.abs(P_e_a[:,j] - P_e_a[:,j-1])
    diagonal = np.zeros((per, per))
    np.fill_diagonal(diagonal, hp_power_el_opt)

    P_e = np.abs(diagonal - P_e)
    P_w = (P_e * M_eta)

    df_per['P_e_opt'] = P_e.sum(1)

    return   M_eta, P_e, P_w, df_per

In [229]:
M_eta, P_e,  P_w, df_per = heating_opt(data_temp)

In [230]:
# maximally possible saving potential, if there was no storage limit
df_per.P_e_opt.sum() / df_per.energy_needed_elec.sum()

1.0337872779518291

## Storage volume

In [231]:
#px.line(temperatures.max(axis=0))

## Visualizations for Optimized Heating

## Energy storage forecast

In [232]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = df_per.date, y = df_per.temperature, name='temperature', mode='lines'),  secondary_y=False)
fig.add_trace(go.Scatter(x = df_per.date, y = df_per.P_e_opt, name='Power (e) optimized', mode='markers'),  secondary_y=True)
fig.add_trace(go.Scatter(x = df_per.date, y = df_per.energy_needed_elec, name='Power (e) not optimized', mode='markers'),  secondary_y=True)
fig.update_layout(title='Power (elec) for optimized and not optimized heating')
fig.write_html('optimization1.html')

In [233]:
# check if optimized power line is always above needed power line
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = df_per.date, y = df_per.energy_needed_water.cumsum(), name='power needed (water)', mode='lines'),  secondary_y=False)
fig.add_trace(go.Scatter(x = df_per.date, y = P_w.sum(1).cumsum(), name='optimized power (water)', mode='lines'),  secondary_y=False)
fig.update_layout(title='Cumulated Energy (water) stored vs energy needed in Watthours')

In [67]:
# check if optimized power line is always above needed power line
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=data_temp_new.date, y=data_temp_new.energy_needed_water.cumsum(), name='power needed (water)', mode='lines'),
    secondary_y=False)
fig.add_trace(go.Scatter(x=data_temp_new.date, y=P_w_new.sum(1).cumsum(), name='optimized power (water)', mode='lines'),
              secondary_y=False)
fig.update_layout(title='Cumulated Energy (water) stored vs energy needed in Watthours')

In [234]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = df_per.date, y = df_per.P_e_opt.cumsum(), name='Power (e) optimized', mode='lines'),  secondary_y=False)
fig.add_trace(go.Scatter(x = df_per.date, y = df_per.energy_needed_elec.cumsum(), name='Power (e) not optimized', mode='lines'),  secondary_y=False)
fig.update_layout(title='Cumulated Power (elec) for optimized and not optimized heating')

In [235]:
# check visually if needed power equals optimized power supply for every period -> lines have to match exactly
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = df_per.date, y = df_per.energy_needed_water.cumsum(), name='power needed (water)', mode='lines'),  secondary_y=False)
fig.add_trace(go.Scatter(x = df_per.date, y = P_w.sum(0).cumsum(), name='optimized power (water)', mode='lines'),  secondary_y=False)

## Heat maps

In [236]:
fig = px.imshow(P_e)
fig.write_html('P_e' + '.html', auto_open=False)

In [237]:
fig = px.imshow(M_eta)
#fig.show()
fig.write_html('M_eta' + '.html', auto_open=False)

In [238]:
fig = px.imshow(P_w)
fig.update_layout(height=1000)
fig.write_html('P_w' + '.html', auto_open=False)

In [239]:
px.line(data_temp.energy_needed_water.rolling(24).sum()).write_html('energy_needed_water' + '.html', auto_open=False)

# Heating optimization with storage limit

In [6]:
# optimize over flowtemp
# TODO divide by period-length (=1, when hourly temperatures)

def heating_optimization(df, period_lim = None, period = None, **kwargs):
    if type(period) != type(None):
        df_per = df[(df.period_max == period) | (df.period_min == period)]
    else:
        df_per = df.copy()

    per = df_per.shape[0] #number of hours in period
    M_timedelta = np.empty((per, per))

    for i in range(per): # build timedelta matrix
        M_timedelta[i, range(0, i+1)] = np.flip(np.linspace(0, i, i+1))
    M_timedelta = M_timedelta.T
    flow = df_per.flow_temp.values

    M_flow = celsius(newton_cooling_t_0(kelvin(flow), M_timedelta * 3600))
    M_flow = np.triu(M_flow) # needed heating temperature when heating in period i and using the heat in period j (i X j - Matrix)
    M_temperature = np.tile(df_per.temperature.values, (len(df_per.temperature.values),1)).T

    def target_fun(fl, optimizing = True, constraint = False):

        M_eta = eta_freezing_adj(zu=M_temperature, max_ab=M_flow)#, min_ab=M_flow)

        if type(period_lim) != type(None): #period_lim also means storage limitation
            per_lim = np.minimum(period_lim, per)
            M_flow_max = celsius(newton_cooling_t_0(kelvin(fl), t=M_timedelta * 3600))
            M_eta_heating = eta_freezing_adj(zu = M_temperature, max_ab=M_flow_max, heating=h)
            M_eta_heating = np.triu(M_eta_heating, k=1)
            M_eta = np.tril(M_eta)

            M_eta = M_eta_heating + M_eta
            M_eta = np.triu(M_eta) # upper diagonal
            M_eta = np.tril(M_eta, k = per_lim)

        else:
            M_eta = np.triu(M_eta) # upper diagonal
        P_w_sum = df_per.energy_needed_water.values # needed energy in water for each hour
        P_e_max = np.full((per, ), hp_power_el_opt) # maximal available energy (electricity) per hour -> e.g. 2500W
        P_e_available = P_e_max.copy() # at each period still available 'capacity' in electricity for future hours

        P_e = np.zeros((per, per)) # Matrix with used energy (e) from each period for each period that is optimized
        P_e_a = np.empty((per, per)) # Matrix to later calculate P_e

        for j in range(per):
            P_w_j = P_w_sum[j] # energy needed (w) in hour j

            M_eta_j = M_eta[: , j] # efficiencies in each hour
            eta_sorted_j_index = np.flip(np.argsort(M_eta_j)) # get indexes of highest values in M_eta_j
            eta_sorted_j = M_eta_j[eta_sorted_j_index] # get highest values of M_eta_j (descending order)

            P_w_available_j = P_e_available[eta_sorted_j_index] * eta_sorted_j # available energy (w) for hour j in descending cost order

            P_w_available_j_cumsum = P_w_available_j.cumsum() # get cumsum for available P_w (descending cost order)
            P_w_available_j_cumsum_shift = P_w_available_j_cumsum - P_w_available_j # shifted cumsum

            # calculate which P_w_available_j values are used fully and which only partly (only one will be used partly)
            cumsum_bin = P_w_available_j_cumsum < P_w_j # binary values if cumsum is smaller than needed energy
            cumsum_shift_bin = P_w_available_j_cumsum_shift < P_w_j # binary values if cumsum_shift is smaller than needed energy

            idx = (cumsum_bin == False) & (cumsum_shift_bin == True) # only True for partly used P_w_j
            percent = (P_w_available_j_cumsum[idx] -P_w_j)  / P_w_available_j[idx] # get remaining share of partly used P_w_j
            P_e_percent_remaining = np.ones((per, )) # first, all remaining values are set to 1 (=100%)
            P_e_percent_remaining[cumsum_bin] = 0 # set all fully used P_w_j to 0
            P_e_percent_remaining[idx] = percent # set partly used to calculated percentage

            P_e_percent_remaining_sorted_orig = P_e_percent_remaining[np.argsort(eta_sorted_j_index)] # set remaining shares to original order
            P_e_available = P_e_available * P_e_percent_remaining_sorted_orig # get new P_e_available
            P_e_a[:,j] = P_e_available

        P_e_a = np.triu(P_e_a) # take only upper diagonal
        P_e[:,0] = P_e_a[:,0]
        for j in range(1, per):
            P_e[:,j] = np.abs(P_e_a[:,j] - P_e_a[:,j-1])
        diagonal = np.zeros((per, per))
        np.fill_diagonal(diagonal, hp_power_el_opt)

        P_e = np.abs(diagonal - P_e)
        P_w = (P_e * M_eta)

        if optimizing == False:
            df_per['P_e_opt'] = P_e.sum(1)
            return M_eta, P_e, P_w, df_per

        if constraint == True:
            return P_w

        return P_e.sum()

    def constraint_fun(fl):

        P_w = target_fun(fl, constraint=True)
        stor_vol = np.zeros((P_w.shape[0],))

        for i in range(P_w.shape[0] - 1):
            stor_vol[i] = get_needed_storage_vol(fl - ruecklauf, P_w[:(i + 1), (i + 1):].sum() / 1000)
        max_stor = max(stor_vol)

        return storage_volume - max_stor

    if 'optimize' in kwargs: #same as heating_opt
        if kwargs['optimize'] == False:
            if 'flow' in kwargs:
                M_eta_opt, P_e_opt, P_w_opt, df_per_opt = target_fun(kwargs['flow'], optimizing=False)
                stor_vol = np.zeros((P_w_opt.shape[0],))

                for i in range(P_w_opt.shape[0] - 1):
                    stor_vol[i] = get_needed_storage_vol(kwargs['flow'] - ruecklauf, P_w_opt[:(i + 1), (i + 1):].sum() / 1000)
                max_stor = max(stor_vol)

                return M_eta_opt, P_e_opt, P_w_opt, df_per_opt
            else:
                return 'Must enter a temperature'

    # optimization problem
    lb = max(flow) + 0.001
    ub = max_heat_cap
    bounds = Bounds(lb=lb, ub=ub)

    #print('lb: ', lb)
    #print('ub: ', ub)

    min = minimize(target_fun, x0=(lb + 1, ), bounds=bounds, constraints={'type':'ineq',
                                                                          'fun': constraint_fun} )
    #print('period_lim:', period_lim)
    #print('minimum x: ', min.x[0])
    #print(min)


    #print()
    M_eta_opt, P_e_opt, P_w_opt, df_per_opt = target_fun(min.x[0], optimizing=False) # get values for optimal heating temperature

    return min, M_eta_opt, P_e_opt, P_w_opt, df_per_opt

# Optimization with varying preheatings

TODO
add P_w to big P_w (=total P_w) with all periods

In [7]:
def optimize_periods(df, max_periods = 36):
    index = 0
    df_new = pd.DataFrame()
    M_eta_new = np.zeros((df.shape[0], df.shape[0]))
    P_e_new = np.zeros((df.shape[0], df.shape[0]))
    P_w_new = np.zeros((df.shape[0], df.shape[0]))
    for p in df.period_max.dropna().unique():
        print('period: ', p)

        min_df = pd.DataFrame(columns=['period_lim', 'min_x', 'min_fun'])

        for i in range(max_periods):
            minimum, M_eta, P_e,  P_w, df_per = heating_optimization(df.loc[index:, :], period=p, period_lim=i)
            if minimum.success == True:
                min_df = min_df.append({'period_lim': i,
                                        'min_x': minimum.x[0],
                                        'min_fun': minimum.fun}, ignore_index=True)


        temp = min_df.loc[np.argmin(min_df.min_fun)].min_x
        per_lim = min_df.loc[np.argmin(min_df.min_fun)].period_lim


        M_eta_opt, P_e_opt,  P_w_opt, df_per_opt = heating_optimization(df.loc[index:, :], period=p, period_lim=per_lim, optimize=False, flow=temp)

        stor_vol = np.zeros((P_w_opt.shape[0],))
        for i in range(P_w_opt.shape[0] - 1):
            stor_vol[i] = get_needed_storage_vol(temp - ruecklauf, P_w_opt[:(i + 1), (i + 1):].sum() / 1000)

        df_per_opt['Storage_vol'] = stor_vol
        df_per_opt['Preheat_temp'] = temp
        df_per_opt['Period_lim'] = per_lim

        df_new = pd.concat([df_new, df_per_opt])
        M_eta_new[index:index+M_eta_opt.shape[0], index:index+M_eta_opt.shape[1]] = M_eta_opt
        P_e_new[index:index+P_e_opt.shape[0], index:index+P_e_opt.shape[1]] = P_e_opt
        P_w_new[index:index+P_w_opt.shape[0], index:index+P_w_opt.shape[1]] = P_w_opt

        index = df_per_opt[df_per_opt.Storage_vol == 0].Storage_vol.index[-1] + 1
    return M_eta_new, P_e_new, P_w_new, df_new

M_eta_new, P_e_new, P_w_new, data_temp_new = optimize_periods(data_temp)

period:  0.0
period:  1.0
period:  2.0
period:  3.0
period:  4.0
period:  5.0
period:  6.0
period:  7.0
period:  8.0
period:  9.0
period:  10.0
period:  11.0
period:  12.0
period:  13.0
period:  14.0
period:  15.0
period:  16.0
period:  17.0
period:  18.0
period:  19.0
period:  20.0
period:  21.0
period:  22.0
period:  23.0
period:  24.0
period:  25.0
period:  26.0
period:  27.0
period:  28.0
period:  29.0
period:  30.0
period:  31.0
period:  32.0
period:  33.0
period:  34.0
period:  35.0
period:  36.0
period:  37.0
period:  38.0
period:  39.0
period:  40.0
period:  41.0
period:  42.0
period:  43.0
period:  44.0
period:  45.0
period:  46.0
period:  47.0
period:  48.0
period:  49.0
period:  50.0
period:  51.0
period:  52.0
period:  53.0
period:  54.0
period:  55.0
period:  56.0
period:  57.0
period:  58.0
period:  59.0
period:  60.0
period:  61.0
period:  62.0
period:  63.0
period:  64.0
period:  65.0
period:  66.0
period:  67.0
period:  68.0
period:  69.0
period:  70.0
period:  71.0
pe

In [9]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data_temp_new.date, y=data_temp_new.temperature, name='temperature', mode='lines'), secondary_y=False)
fig.add_trace(go.Scatter(x=data_temp_new.date, y=data_temp_new.P_e_opt, name='Power (e) optimized', mode='markers'), secondary_y=True)
fig.add_trace(go.Scatter(x=data_temp_new.date, y=data_temp_new.energy_needed_elec, name='Power (e) not optimized', mode='markers'),
              secondary_y=True)
fig.add_trace(go.Scatter(x=data_temp_new.date, y=data_temp_new.Storage_vol, name='Storage Volume'), secondary_y=True)
fig.add_trace(go.Scatter(x=data_temp_new.date, y=data_temp_new.Preheat_temp, name='Preheat Temperature'), secondary_y=False)
fig.add_trace(go.Scatter(x=data_temp_new.date, y=data_temp_new.flow_temp, name='flowtemperatur'), secondary_y=False)
fig.update_layout(title='Power (elec) for optimized and not optimized heating')
fig.write_html('optimization_data_temp.html', auto_open = False)


In [8]:
data_temp_new.P_e_opt.sum() / data_temp_new.energy_needed_elec.sum()

0.8478194615281115

In [244]:
fig = px.imshow(P_e_new)
fig.write_html('P_e' + '.html', auto_open=False)
fig = px.imshow(M_eta_new)
fig.write_html('M_eta' + '.html', auto_open=False)
fig = px.imshow(P_w_new)
fig.write_html('P_w' + '.html', auto_open=False)